In [1]:
!pip install beautifulsoup4

In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def logger(*args):
    """
        Print log message with excecuting file name, timestamp.
        *** All parameters should be on string-type!
    """
    from datetime import datetime
    try: __file__
    except NameError:
        message = f"[{str(datetime.now())}] {str(' '.join(args))}"
    else:
        message = f"[{__file__} {str(datetime.now())}] {str(' '.join(args))}"
    print(message)

def parsingFrom(url):
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        logger(f"Parsing done successfully from {url}")
        return soup
    else: 
        print(response.status_code)

logger('Done!')


[2021-06-22 12:09:19.097191] Done!


In [36]:
targetURL = "https://www.naturing.net/o/thumbnail?observation_name=%EB%B6%89%EC%9D%80%EA%B7%80%EA%B1%B0%EB%B6%81&order_code=&habitat_code=&area=&help_name_yn=&match_case=N&observe_date1=&observe_date2=&create_date1=&create_date2=&obs_filter=0&follow_yn=&media_type=photo%2Cvideo%2Csound&type=&draft_yn=&sort_type=&lat1=&lng1=&lat2=&lng2="

soup = parsingFrom(targetURL)

[2021-06-22 12:09:26.659897] Parsing done successfully from https://www.naturing.net/o/thumbnail?observation_name=%EB%B6%89%EC%9D%80%EA%B7%80%EA%B1%B0%EB%B6%81&order_code=&habitat_code=&area=&help_name_yn=&match_case=N&observe_date1=&observe_date2=&create_date1=&create_date2=&obs_filter=0&follow_yn=&media_type=photo%2Cvideo%2Csound&type=&draft_yn=&sort_type=&lat1=&lng1=&lat2=&lng2=


In [37]:
table = soup.find_all("div", class_="b-itemlist__item observation-item")

linkHead = "https://www.naturing.net/o/"
linkList = list()

for data in table:
    link = data.find('a')['href']
    linkList.append(link)
else:
    logger(str(len(linkList)), "links parsed.")

[2021-06-22 12:11:31.150331] 160 links parsed.


In [102]:
columnList = ['id', 'address', 'timesstamp', 'latitude', 'longtitude']
result = pd.DataFrame(columns=columnList)

for linkTail in linkList:
    link = linkHead + linkTail
    soup = parsingFrom(link)

    mapPath = soup.select_one('#map')
    if mapPath is None:
        continue
    else:
        latitude, longtitude = mapPath['data-lat'], mapPath['data-lng']

    information = soup.find('ul', class_="obsinfo__list").find_all('div', class_="obsinfo__item-content")
    address = information[0].get_text().strip()
    timestamp = information[3].get_text().strip()
    
    logger(f'id: {linkTail}, address: {address}, timestamp: {timestamp}, latitude: {latitude}, longtitude: {longtitude}')

    dataList = [linkTail, address, timestamp, latitude, longtitude]
    result = result.append({col:data for col, data in zip(columnList, dataList)}, ignore_index=True)
else:
    display(result)
    logger('Done!')

.092477] id: 536342, address: 울산 중구 다운동, timestamp: 2020년 6월 9일 오전 11시, latitude: 35.5624972403092, longtitude: 129.25923014853356
[2021-06-22 13:04:23.661784] Parsing done successfully from https://www.naturing.net/o/535606
[2021-06-22 13:04:23.663501] id: 535606, address: 서울 노원구 상계동, timestamp: 2020년 6월 9일 오전 8시, latitude: 37.67417689972223, longtitude: 127.04943909999997
[2021-06-22 13:04:24.239579] Parsing done successfully from https://www.naturing.net/o/527863
[2021-06-22 13:04:24.241324] id: 527863, address: 서울 광진구 능동, timestamp: 2020년 6월 1일 오후 12시, latitude: 37.548077321749595, longtitude: 127.0784893061729
[2021-06-22 13:04:24.789353] Parsing done successfully from https://www.naturing.net/o/527507
[2021-06-22 13:04:24.790994] id: 527507, address: 부산 남구 용호동, timestamp: 2020년 5월 31일 오전 9시, latitude: 35.10362269616284, longtitude: 129.12328076914187
[2021-06-22 13:04:25.341200] Parsing done successfully from https://www.naturing.net/o/525800
[2021-06-22 13:04:25.342812] id: 5258

,id,address,timesstamp,latitude,longtitude
0,843864,부산 연제구 연산동,2021년 6월 6일 오후 3시,35.19073939972222,129.10276569972223
1,843789,부산 연제구 연산동,2021년 4월 13일 오전 9시,35.19292269972222,129.09424219972223
2,839255,인천 남동구 고잔동,2021년 6월 12일 오후 12시,37.39163932683185,126.67556223353148
3,834991,충북 청주시 서원구 산남동,2021년 6월 8일 오후 3시,36.613734,127.468474
4,833971,서울 송파구 방이동,2021년 6월 2일 오전 9시,37.511819702472465,127.13872625499383
...,...,...,...,...,...
144,24592,부산광역시 사하구 하단1동,2016년 4월 3일 오후 2시,35.10439385332337,128.94590664978057
145,16009,전라북도 전주시 덕진구 송천1동,2015년 9월 5일 오전 9시,35.86139520677234,127.1366323367414
146,12287,경상남도 통영시 산양읍,2015년 7월 8일 오후 4시,34.80383055555555,128.42108055555553
147,10431,부산광역시 사상구 삼락동,2015년 5월 31일 오후 12시,35.17226788888889,128.97369383333333


[2021-06-22 13:05:12.766896] Done!


In [104]:
result.to_csv('data.csv', index=False)